<a href="https://colab.research.google.com/github/Sonic11-stack/face-recognizer/blob/main/Own_Interface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip

In [3]:
!unzip -q ngrok-stable-linux-amd64.zip

In [4]:
!./ngrok authtoken 2Zd7U1RnDwX4FI8W04u7Y1c1e1t_3wnBcLsFJMKxdgQfMzyG3

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [5]:
def get_link():
    get_ipython().system_raw('./ngrok http 8501 &')
    !curl -s http://localhost:4040/api/tunnels | python3 -c \
    'import sys, json; print("Ссылка для streamlit: " +json.load(sys.stdin)["tunnels"][0]["public_url"])'

In [6]:
!pip -q install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00


In [7]:
code = '''
import os
from zipfile import ZipFile
import streamlit as st
from PIL import Image
import os

from tensorflow.keras.models import load_model

model_dict = load_model('model_weights.h5')
from zipfile import ZipFile
work_dir = os.getcwd()                                                  #Сохраняет текущий рабочий каталог.
with ZipFile(os.path.join(work_dir ,'/content/drive/MyDrive/Collab Notebooks/archive_1.zip'),'r') as zipobject:
#with ZipFile(os.path.join(work_dir ,'archive.zip'),'r') as zipobject:
  zipobject.extractall()                                                #Распакование загруженного набора данных.

train_dir = os.path.join(work_dir,'train')       #Путь к каталогу train.
test_dir = os.path.join(work_dir,'test')         #Путь к тестовому каталогу.
os.makedirs(train_dir, exist_ok=True)  #Создание каталога для обучающих примеров.
os.makedirs(test_dir, exist_ok=True)   #Создание каталога для тестирования примеров.
train_ratio=0.8
test_ratio=0.2
dir_list=os.listdir(os.path.join(work_dir ,'105_classes_pins_dataset'))
print('The Number of Classes in the Dataset is:{}'.format(len(dir_list)))
#Этот код разделит распакованный набор данных на обучающий и тестовый наборы в соответствии со значениями train_ratio и test_ratio.
from shutil import copyfile
import numpy as np

source_dir = os.path.join(work_dir ,'105_classes_pins_dataset')        #Распакованный каталог набора данных

dir_list = os.listdir(source_dir)

for folder in dir_list:
    data_dir = os.listdir(os.path.join(source_dir,folder))
    np.random.shuffle(data_dir)                            #Перетасовка примеров классов внутри набора данных.
    os.makedirs(os.path.join(train_dir , folder), exist_ok=True)
    os.makedirs(os.path.join(test_dir , folder), exist_ok=True)
    train_data = data_dir[:int(len(data_dir)*train_ratio+1)] #Разделение обучающего набора данных по отношению к train_ratio.
    test_data = data_dir[-int(len(data_dir)*test_ratio):]

    for image in train_data:
        copyfile(os.path.join(source_dir,folder,image) , os.path.join(train_dir,folder,image)) #Копирование обучающих файлов из набора данных в каталог обучения.

    for image in test_data:
       copyfile(os.path.join(source_dir,folder,image) , os.path.join(test_dir,folder,image))  #Копирование обучающих файлов из набора данных в каталог обучения.

print("The Number of Classes in the Training Set:{}".format(len(os.listdir(train_dir))))
print("The Number Of Classes in the Testing Set:{}".format(len(os.listdir(test_dir))))
print('For Verification of the Split.....')
print("The Number of Examples in the Class='pins_Katherine Langford' in the Dataset Directory:{}".format(len(os.listdir(os.path.join(source_dir,'pins_Katherine Langford')))))
print("The Number of Examples in the Class='pins_Katherine Langford' in the Train Directory:{}".format(len(os.listdir(os.path.join(train_dir,'pins_Katherine Langford')))))
print("The Number of Examples in the Class='pins_Katherine Langford' in the Test Directory:{}".format(len(os.listdir(os.path.join(test_dir,'pins_Katherine Langford')))))
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
rows=160  #Количество пикселей в строках для ввода.
cols=160  #Количество пикселей в столбцах для вывода.
trained_model=MobileNetV2(input_shape=(rows,cols,3),
                    include_top=False,
                    weights='imagenet')


trained_model.trainable=True  #Разморозка всех предварительно подготовленных слоев Mobile Net V2 для обучения.

trained_model.summary()
last_layer=trained_model.get_layer('out_relu')
last_layer_output=last_layer.output  #Сохраняет выходные данные последнего уровня Mobile Net V2.
#Функция обратного вызова, которая прекращает обучение, когда точность достигает 98%.
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.98):
      self.model.stop_training = True
callbacks=myCallback()
x = tf.keras.layers.GlobalAveragePooling2D()(last_layer_output)

x = tf.keras.layers.Dropout(0.8)(x)

x = tf.keras.layers.Dense(105,activation='softmax')(x)

model = tf.keras.Model(trained_model.input,x)

model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])
#Краткое описание модели.
model.summary()
train_datagen = ImageDataGenerator(rescale=1/255,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 rotation_range=40,
                                 width_shift_range=0.1,
                                 height_shift_range=0.1) # создание генератора обучающих данных (train_datagen) с использованием класса ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1/255)

train_generator=train_datagen.flow_from_directory(train_dir,
                                                  target_size=(rows,cols),
                                                  class_mode='categorical') #метод загружает изображения из указанной директории и применяет определенные трансформации

test_generator=test_datagen.flow_from_directory(test_dir,
                                                target_size=(rows,cols),
                                                class_mode='categorical') # Аналогично создается генератор тестовых данных (test_generator) для директории тестовых данных


st.title('Face Recognition demo')

image_file = st.file_uploader('Load an image', type=['png', 'jpg'])
if not image_file is None:
    classes=train_generator.class_indices
    class_names=list(classes.keys())   #Список имен классов.
    img = tf.keras.preprocessing.image.load_img(
        image_file, target_size=(rows, cols)
    )

    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array=img_array/255.

    score = model_dict.predict(img_array)

    image = Image.open(image_file)
    st.image(image, caption='Uploaded Image.', use_column_width=True)
    st.write("The Person in the Image is {} ."
    .format(class_names[np.argmax(score)][5:].title()))
'''

with open('hello.py', 'w') as f:
    f.write(code)

In [9]:
get_link()

Ссылка для streamlit: https://ab58-35-229-187-174.ngrok-free.app


In [10]:
!streamlit run hello.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://35.229.187.174:8501

2023-12-25 09:56:22.751194: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-25 09:56:22.751270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-25 09:56:22.754190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-25 09:56:22.769795: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions